In [ ]:
import pandas as pd
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import itertools
import numpy as np
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#load data
DATA_PATH1 = '/content/drive/My Drive/Colab Notebooks/chromium.csv'
data1 = pd.read_csv(DATA_PATH1, sep='\t')
data_top1 = data1.head()
print("Chromium Data")
data_top1

Chromium Data


,title,description,type
0,Define a way to enforce performance constraint...,\nThere are cases where tests should fail if a...,Bug
1,Websites that don't work with Chrome Browser,\nChrome Version : 3.0.195.10\r\nURLs (i...,Compat
2,Redraw issue/truncation with confirmation item...,\nChromium 0.3.155.0 (Developer Build 3546)\r\...,Bug
3,Implement CanvasRenderingContext2D for access ...,\nApplications that do background image proces...,Feature
4,ath9k: panic in ath_tx_last_beacon + 0x29a,\nToT (3.0 kernel) panic'd on resume. Was ass...,Bug


In [ ]:
DATA_PATH2 = '/content/drive/My Drive/Colab Notebooks/linux_bugs_usage_ready.csv'
data2 = pd.read_csv(DATA_PATH2, sep='\t')
data_top2 = data2.head()
print("Linux Data")
data_top2

Linux Data


,title,message,importance,product,component
0,oops when using ide-cd with 2.5.45 and cdrecord,Please enter Exact Kernel version:2.5.45 Distr...,P2 normal,IO/Storage,IDE
1,NUMA-Q hangs during TSC initialization on boot.,Exact Kernel version: 2.5.46 Distribution: deb...,P2 normal,Platform Specific/Hardware,i386
2,Enabling shared pagetables causes KDE to wierd...,Exact Kernel version: 2.5.46-mm1 Distribution:...,P2 normal,Memory Management,Other
3,Dcache spirals out of control on 2.5.43-mm2,Exact Kernel version: 2.5.43-mm2 Distribution:...,P2 blocking,IO/Storage,Other
4,64GB highmem BUG(),Exact Kernel version: 2.5.40 Hardware Environm...,P2 normal,Memory Management,Other


In [ ]:
#merge title and message
def merge_title_and_message(data, message_col_name='message'):
  data['text'] = data['title'] + ' ' + data[message_col_name]
  data = data.drop(['title'], axis=1)
  data = data.drop([message_col_name], axis=1)
  return data
data1 = merge_title_and_message(data1, message_col_name='description')
data_top1 = data1.head()
print("Chromium Merge Title and Message Data")
data_top1

Chromium Merge Title and Message Data


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that don't work with Chrome Browser \...
2,Bug,Redraw issue/truncation with confirmation item...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k: panic in ath_tx_last_beacon + 0x29a \nT...


In [ ]:
data2 = merge_title_and_message(data2)
data_top2 = data2.head()
print("Linux Merge Title and Message Data")
data_top2

Linux Merge Title and Message Data


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using ide-cd with 2.5.45 and cdrecor...
1,P2 normal,Platform Specific/Hardware,i386,NUMA-Q hangs during TSC initialization on boot...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2.5.43-mm2 Ex...
4,P2 normal,Memory Management,Other,64GB highmem BUG() Exact Kernel version: 2.5.4...


In [ ]:
def strip_punctuations(data, column_name='text'):
  '''
  Strips punctuations from the end of each token.
  This uses suggestion from https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate
  to accomplish this really fast.
  '''
  translator = str.maketrans('', '', string.punctuation)
  data['text'] = data['text'].map(lambda s : str(s).translate(translator))
  return data
data1 = strip_punctuations(data1)
data_top1 = data1.head()
print("Chromium strip punctuations")
data_top1

Chromium strip punctuations


,type,text
0,Bug,Define a way to enforce performance constraint...
1,Compat,Websites that dont work with Chrome Browser nC...
2,Bug,Redraw issuetruncation with confirmation item ...
3,Feature,Implement CanvasRenderingContext2D for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a nToT 30 ...


In [ ]:
data2 = strip_punctuations(data2)
data_top2 = data2.head()
print("Linux strip punctuations")
data_top2

Linux strip punctuations


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on 2543mm2 Exact...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


In [ ]:
def remove_linux_garbage(data):
  '''
  Linux data contains lots of garbage, e.g. memory addresses - 0000f800
  '''
  def is_garbage(w):
    return len(w) >= 7 and sum(c.isdigit() for c in w) >= 2

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if not is_garbage(w) else ' ', s.split())))
  return data
data = remove_linux_garbage(data2)
data_top2 = data2.head()
print("Remove linux garbage")
data_top2

Remove linux garbage


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord P...
1,P2 normal,Platform Specific/Hardware,i386,NUMAQ hangs during TSC initialization on boot ...
2,P2 normal,Memory Management,Other,Enabling shared pagetables causes KDE to wierd...
3,P2 blocking,IO/Storage,Other,Dcache spirals out of control on Exact Kerne...
4,P2 normal,Memory Management,Other,64GB highmem BUG Exact Kernel version 2540 Har...


In [ ]:
def cast_to_lowercase(data):
  data['text'] = data['text'].map(lambda s : s.lower())
  return data
data1 = cast_to_lowercase(data1)
data_top1 = data1.head()
print("Chromium Lowercase")
data_top1

Chromium Lowercase


,type,text
0,Bug,define a way to enforce performance constraint...
1,Compat,websites that dont work with chrome browser nc...
2,Bug,redraw issuetruncation with confirmation item ...
3,Feature,implement canvasrenderingcontext2d for access ...
4,Bug,ath9k panic in athtxlastbeacon 0x29a ntot 30 ...


In [ ]:
data2 = cast_to_lowercase(data2)
data_top2 = data2.head()
print("Linux Lowercase")
data_top2

Linux Lowercase


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops when using idecd with 2545 and cdrecord p...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs during tsc initialization on boot ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde to wierd...
3,P2 blocking,IO/Storage,Other,dcache spirals out of control on exact kerne...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


In [ ]:
def remove_stopwords(data):
  stop_words = stopwords.words('english')
  translator = str.maketrans('', '', string.punctuation)
  stop_words = set([w.translate(translator) for w in stop_words]) # Apostrophes were removed already

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if w not in stop_words else ' ', s.split())))
  return data
data1 = remove_stopwords(data1)
data_top1 = data1.head()
print("Chromium Stopword")
data_top1

Chromium Stopword


,type,text
0,Bug,define way enforce performance constraint ...
1,Compat,websites work chrome browser nchrome ver...
2,Bug,redraw issuetruncation confirmation item d...
3,Feature,implement canvasrenderingcontext2d access ...
4,Bug,ath9k panic athtxlastbeacon 0x29a ntot 30 ke...


In [ ]:
data2 = remove_stopwords(data2)
data_top2 = data2.head()
print("Linux Stopword")
data_top2

Linux Stopword


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please en...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact ...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ...
3,P2 blocking,IO/Storage,Other,dcache spirals control exact kernel vers...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


In [ ]:
def remove_rare_words(data, min_count=3):
  wc = {} # WordCount
  def proc_word(s):
    for w in set(s.split()):
      if w in wc:
        wc[w] += 1
      else:
        wc[w] = 1

  for index, row in data.iterrows():
    proc_word(row['text'])

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if wc[w] >= min_count else ' ', s.split())))
  return data
data1 = remove_rare_words(data1)
data_top1 = data1.head()
print("Chromium rare words")
data_top1

Chromium rare words


,type,text
0,Bug,define way enforce performance constraint spec...
1,Compat,websites work chrome browser nchrome version ...
2,Bug,redraw confirmation item download bar nchrom...
3,Feature,implement canvasrenderingcontext2d access work...
4,Bug,ath9k panic ntot 30 kernel resume associ...


In [ ]:
data2 = remove_rare_words(data2)
data_top2 = data2.head()
print("Linux rare words")
data_top2

Linux rare words


,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please enter ex...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact kern...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ex...
3,P2 blocking,IO/Storage,Other,dcache control exact kernel version distribu...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...
